# web scraping Listings usings BeautifulSoup


In [ ]:
import requests
from sklearn.utils import shuffle
import json
import pandas as pd
import time
from bs4 import BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_real_estate_data(url,rent=None):
    # Send an HTTP GET request to the URL
    response = requests.get(url)
    
    # Check if the request was successful
    print(response.status_code)
    
    
    
    
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract property listings
        listings = soup.find_all('div', class_='lazyload-wrapper')
        
        
        
        properties = []
        # Extract data from each listing
        for listing in listings:
            # Extract property details (e.g., price, location, etc.)
            link_tag = listing.find('div', class_='results_card')
            link = link_tag.find('a')['href']
            
            price = listing.find('span', class_='price').text
            if rent:
                _ ,price,_, _ = price.replace("\xa0"," ").split(" ")
            else:
                try:
                    _ ,price = price.strip().replace("\xa0"," ").strip().split(" ")
                    
                except:
                    price = "0,"
                
            title_location_div = listing.find('div', class_='card_title')
            title=title_location_div.find("h2").text.strip()
            location=title_location_div.find("h5").text.strip()
            
            # Extract image URL if available
            image_url_div = listing.find('div', class_='results_card_media')
            image_url = image_url_div.find_all('img')[2]['src'] 

            # desciptions
            desc_tag = listing.find('ul',class_='results_card_details')            
            desc = ""
            
            # Extract the text of the <li> tag and concatenate it with the existing descriptions
            for li_tag in desc_tag.find_all( ['li','p'] ):
                desc +=  li_tag.text + " | "
                
            response_click=requests.get(link)
            if response_click.status_code == 200:
                soup_click = BeautifulSoup(response_click.text,"html.parser")
                sub_listings = soup_click.find("div",class_="tower_description")
                description = sub_listings.find("p").text.strip() if sub_listings.find("p") else ""
                summary = soup_click.find("div",class_="tower_detail").find("h1").text
                
                
            else:
                description=""

                
            # get agents contact
            
            ag_name = listing.find('div',class_='agent_name').find('span').text
            
            ag_whatsapp = listing.find('div',class_='agent_contact').find('a')['href']
            ag_call = listing.find('div',class_='agent_contact').find_all('a')[1]['href']
            
            #get the bed and = bath
            
            bed_bath_div = listing.find("div",class_="about_tour")
            bed , bath , sqft = bed_bath_div.find_all("span")  # this will be 3 list 0-bed 1-bath 2-hight(not using it)
            sqft, _ = sqft.text.split(" ")
            number_of_beds =int(bed.text) if  bed  else 0
            number_of_baths = int(bath.text) if  bath.text.isdigit()  else 0
            size_in_sqft = int(sqft.replace(",","")) if sqft else 0
            
            
            
            properties.append ({
                'title': title,
                'description': desc,
                'price_in_aed': int(price.replace(",","")), 
                'location': location,
                'number_of_beds': number_of_beds,
                'number_of_baths':number_of_baths ,
                "size_in_sqft": size_in_sqft,
                'sale_type': "rent" if rent else "buy",
                'page_url':link,
                'image_url': image_url,
                'agent_name':ag_name,
                'whatsapp_link':ag_whatsapp,
                'call_agent':ag_call,
                'long_description': description,
                "summary":summary
                
            })
        
        return properties
    else:
        print("Failed to fetch the page")



In [ ]:
# ## NOW LETS GRAP SOME MORE PAGES AND STORE TO DF


import numpy as np
from sklearn.utils import shuffle

num_pages = 20
starter = 0
rent_listings= []
sale_listings= []
st = time.time()
for _ in range(2):
    rent=True
    
    for i in range(num_pages):
        url_rent= f"https://www.allsoppandallsopp.com/dubai/properties/residential/lettings/page-{starter+i+1}"
        listings = scrape_real_estate_data(url_rent,rent)
        rent_listings += listings

        time.sleep(30)
        print("rent_page : ", f"{starter+i+1}/40")


    time.sleep(300)

    for i in range(num_pages):
        url_sale= f"https://www.allsoppandallsopp.com/dubai/properties/residential/sales/page-{starter+i+1}"
        listings = scrape_real_estate_data(url_sale)
        sale_listings += listings

        time.sleep(30)

        print("sale_page : ", starter+i+1,"/40")
            


        time.sleep(300)
        starter = starter+20
end= time.time()

print("rantime: {end-st}")

rent_json=rent_listings
sale_json=sale_listings

rent_df = pd.DataFrame(rent_json)
sale_df = pd.DataFrame(sale_json)


df_final = shuffle(pd.concat([rent_df,sale_df]))



In [ ]:
property_df = df_final
columns_to_separate=["page_url","image_url","call_agent","whatsapp_link","agent_name"] 
property_df.reset_index(inplace=True,drop=True)

property_df = property_df[~property_df.index.isin([51, 71])]  ## contains np.nan // unusable listings
extras= property_df[columns_to_separate]

property_df.drop(columns_to_separate ,axis=1,inplace=True)

property_df.reset_index(inplace=True,drop=True)

extras.reset_index(inplace=True,drop=True)

ids = pd.DataFrame({"id":list(range(1,len(extras) + 1 ))})
                   
extras = pd.concat([ids,extras],axis=1)
property_df = pd.concat([ids, property_df],axis=1)

property_df["id"][8]==9

In [ ]:
def remove_after_please(text):
    parts = text.split("Please")  ## we have plese call and note
    return parts[0].strip() if len(parts) > 1 else text



# Apply the function to the 'description' column
property_df['long_description'] = property_df['long_description'].apply(remove_after_please)
                                                              
for _ in range(10):
    property_df.long_description = (property_df.long_description+" in "
                                    +property_df.location +" price "
                                    +property_df.price_in_aed.astype(str) +" aed "
                                    +" "
                                    +property_df.number_of_beds.astype(str)+" bedrooms for ")
    property_df.long_description += property_df.sale_type if all(property_df.sale_type) =="rent" else "sale"
    
    

In [ ]:
# save the results to csv

property_df.to_csv("estate_data.csv")

EXTRAS = pd.concat([extras,property_df.description,property_df.title,property_df.location,property_df.sale_type,property_df.number_of_beds,property_df.price_in_aed],axis=1)
EXTRAS.to_csv("contact_data.csv")